In [2]:
import pandas as pd
import epiweeks
import datetime
import numpy as np

## LOAD DATA

In [8]:
#Load surveillance Data
#surveillance data
locations = pd.read_csv("./dat/locations.csv")

surv = pd.read_csv("./COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW_.csv")
surv['date'] = pd.to_datetime(surv['date'])
surv['hosp_inc'] = surv['previous_day_admission_adult_covid_confirmed']+ surv['previous_day_admission_pediatric_covid_confirmed']
surv['new_date'] = surv['date']-datetime.timedelta(days=1)
surv = surv[['state','new_date','hosp_inc']]
surv = surv.merge(locations[['abbreviation','location']], left_on ="state",right_on="abbreviation")
surv['week_end_date'] = surv['new_date'].apply(lambda key: epiweeks.Week.fromdate(key).cdcformat())
surv['date'] = pd.to_datetime(surv['new_date'].apply(lambda key: epiweeks.Week.fromdate(key).enddate()))

surv_weekly = surv.groupby(['state','location','week_end_date','date'])['hosp_inc'].sum().reset_index()
surv_weekly = surv_weekly[~surv_weekly.state.isin(['PR',"VI","AS"])]
US = surv_weekly.groupby(['week_end_date','date'])['hosp_inc'].sum().reset_index()
US['state'] = "US"
US['location'] = "US"
surv_weekly = pd.concat([surv_weekly,US[['state','location','week_end_date','date','hosp_inc']]])
surv_weekly = surv_weekly[surv_weekly.date<=datetime.datetime(2023,8,26)]
surv_weekly = surv_weekly[surv_weekly.date>=datetime.datetime(2023,4,16)]

In [36]:
surv_weekly[surv_weekly.location =='56']

,state,location,week_end_date,date,hosp_inc
9857,WY,56,202316,2023-04-22,15.0
9858,WY,56,202317,2023-04-29,13.0
9859,WY,56,202318,2023-05-06,15.0
9860,WY,56,202319,2023-05-13,9.0
9861,WY,56,202320,2023-05-20,20.0
9862,WY,56,202321,2023-05-27,12.0
9863,WY,56,202322,2023-06-03,22.0
9864,WY,56,202323,2023-06-10,17.0
9865,WY,56,202324,2023-06-17,10.0
9866,WY,56,202325,2023-06-24,15.0


In [17]:
incidence = True
target_obs = 'hospitalization'
observations = pd.read_parquet(f"./dat/truth_{'inc' if incidence else 'cum'}_{target_obs}.pq")
observations['date'] = pd.to_datetime(observations['date'])

observations = observations[(observations['date'] >= datetime.datetime(2023,4,16)) & \
               (observations['date'] <= datetime.datetime(2023,8,26))]

observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()


In [34]:
observations[observations.location == '37']

,location,date,value
627,37,2023-04-22,221
628,37,2023-04-29,165
629,37,2023-05-06,189
630,37,2023-05-13,166
631,37,2023-05-20,176
632,37,2023-05-27,141
633,37,2023-06-03,156
634,37,2023-06-10,114
635,37,2023-06-17,145
636,37,2023-06-24,119


In [3]:
#Load Round 17 Data
round17_file = "/Users/davis.jes/Documents/GitHub/covid19-scenario-modeling-hub/data-processed/MOBS_NEU-GLEAM_COVID/2023-04-16-MOBS_NEU-GLEAM_COVID.gz.parquet"
data = pd.read_parquet(round17_file)
data= data[(data['type']=="sample") & (data['target']=="inc hosp") & (data['horizon']<=104)]

start_week = epiweeks.Week.fromdate(datetime.datetime(2023,4,16))
data['start_week'] = start_week
data['week_end_date'] = data['start_week'] + (data['horizon']-1).astype(int)

#Load surveillance Data
#surveillance data
locations = pd.read_csv("./locations.csv")

surv = pd.read_csv("/Users/davis.jes/Downloads/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries__RAW_.csv")
surv['date'] = pd.to_datetime(surv['date'])
surv['hosp_inc'] = surv['previous_day_admission_adult_covid_confirmed']+ surv['previous_day_admission_pediatric_covid_confirmed']
surv['new_date'] = surv['date']-datetime.timedelta(days=1)
surv = surv[['state','new_date','hosp_inc']]
surv = surv.merge(locations[['abbreviation','location']], left_on ="state",right_on="abbreviation")
surv['week_end_date'] = surv['new_date'].apply(lambda key: epiweeks.Week.fromdate(key).cdcformat())
surv['date'] = pd.to_datetime(surv['new_date'].apply(lambda key: epiweeks.Week.fromdate(key).enddate()))

surv_weekly = surv.groupby(['state','location','week_end_date','date'])['hosp_inc'].sum().reset_index()
surv_weekly = surv_weekly[~surv_weekly.state.isin(['PR',"VI","AS"])]
US = surv_weekly.groupby(['week_end_date','date'])['hosp_inc'].sum().reset_index()
US['state'] = "US"
US['location'] = "US"
surv_weekly = pd.concat([surv_weekly,US[['state','location','week_end_date','date','hosp_inc']]])
surv_weekly = surv_weekly[surv_weekly.date<=datetime.datetime(2023,8,26)]


data['date'] = data['week_end_date'].apply(lambda key: key.enddate())
data['date'] = pd.to_datetime(data['date'])
data['type_id'] = data['type_id'].astype(int)
data = data[data.date<= datetime.datetime(2023,8,26)]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/davis.jes/Documents/GitHub/covid19-scenario-modeling-hub/data-processed/MOBS_NEU-GLEAM_COVID/2023-04-16-MOBS_NEU-GLEAM_COVID.gz.parquet'

# Calculating the energy score

In [91]:
#calculate first part of equation
merged = data.merge(surv_weekly, on = ['location','date'], suffixes = ["_model","_surv"])
merged['squared_diff'] = (merged['value']-merged['hosp_inc'])**2

part1 = merged.groupby(['scenario_id','location','type_id'])['squared_diff'].agg([sum,len]).reset_index().rename(columns = {"len":"M"})
part1['sqrt_sum'] = np.sqrt(part1['sum'])
part1_final = part1.groupby(['scenario_id','location'])['sqrt_sum'].agg([sum,len]).reset_index().rename(columns = {"len":"N"})
part1_final['part1'] = part1_final['sum']/part1_final['N']


#calculate the second part of the equation
data['key'] = 1
cross_join = data.merge(data, on = ['scenario_id','location','date','key'], suffixes = ["_i","_j"])
cross_join['squared_diff'] = (cross_join['value_i']-cross_join['value_j'])**2

part2 = cross_join.groupby(['scenario_id','location','type_id_i','type_id_j'])['squared_diff'].agg([sum,len]).reset_index().rename(columns = {"len":"M"})
part2['sqrt_sum'] = np.sqrt(part2['sum'])
part2_final = part2.groupby(['scenario_id','location'])['sqrt_sum'].agg([sum,len]).reset_index().rename(columns = {"len":"N_combo"})
N=100
part2_final['part2'] = part2_final['sum']/(2*N**2)

final = part2_final.merge(part1_final, on = ['scenario_id','location'])
final['ES'] = final['part1']-final['part2']

In [92]:
df_BQ = pd.read_csv("test_ES_team_locations_0422-0826_Scenario-A.csv")
df_mobs = df_BQ[df_BQ.team == "MOBS_NEU-GLEAM_COVID"]

In [93]:
final_merged = df_mobs.merge(final, on = ['scenario_id','location'])
final_merged['diff'] = abs(final_merged["ES_x"]-final_merged['ES_y'])

In [94]:
final_merged['diff'].sum()

1.1574741165532032e-11

In [95]:
final_merged[abs(final_merged['diff'])>10**-9]

,scenario_id,location,team,state,part1_sum,num_runs,part2_sum,ES_x,sum_x,N_combo,part2,sum_y,N,part1,ES_y,diff


In [96]:
data.horizon.min()

1